In [2]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import pandas as pd
from bertviz import head_view
import torch
from torch import nn
from sklearn.model_selection import train_test_split
import tqdm
import pickle
import numpy as np

/tmp/ipykernel_240326/860917103.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
path_dev = "../data/labeled-train.model-aware.v2.json"
df = pd.read_json(path_dev)

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer_config.json: 100%|██████████| 564/564 [00:00<00:00, 312kB/s]
sentencepiece.bpe.model: 100%|██████████| 4.85M/4.85M [00:00<00:00, 12.9MB/s]
tokenizer.json: 100%|██████████| 17.3M/17.3M [00:01<00:00, 9.36MB/s]
special_tokens_map.json: 100%|██████████| 3.55k/3.55k [00:00<00:00, 20.5MB/s]


In [72]:
df[df["task"] == "MT"].iloc[2].to_dict()

{'hyp': 'Separation and reintegration continue to shift the differences between the two pools with each generation.',
 'ref': 'either',
 'src': 'अलगाव आ पुनर्संयोजन प्रत्येक पीढ़ीक संग द्विटा पूल सभक बीच फेरबदल भिन्नता कें आगू-पाछू करैत रहैत अछि।',
 'tgt': 'Segregation and recombination shuffle variation back and forth between the two pools with each generation.',
 'model': 'facebook/nllb-200-distilled-600M',
 'task': 'MT',
 'labels': ['Not Hallucination',
  'Hallucination',
  'Hallucination',
  'Not Hallucination',
  'Hallucination'],
 'label': 'Hallucination',
 'p(Hallucination)': 0.6000000000000001}

In [73]:
src = df[df["task"] == ""].iloc[2]["src"]
src

'अलगाव आ पुनर्संयोजन प्रत्येक पीढ़ीक संग द्विटा पूल सभक बीच फेरबदल भिन्नता कें आगू-पाछू करैत रहैत अछि।'

In [54]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

In [74]:
tokens = tokenizer(src,return_tensors="pt")

In [87]:
outputs = model.generate(**tokens, 
                         forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
                         num_beams=10,
                         do_sample=True,
                         num_return_sequences=1)
text = tokenizer.batch_decode(outputs)
text

['</s>eng_Latn Separation and reintegration continues to make a difference between the two pools from one generation to the next.</s><pad>',
 '</s>eng_Latn Separation and reintegration continues to make a distinction between the two pools from one generation to the next.</s>',
 '</s>eng_Latn Separation and reintegration continues to make a difference between the two pools with each generation.</s><pad><pad><pad><pad>',
 '</s>eng_Latn Separation and reintegration continues to make a difference between the two pools from generation to generation.</s><pad><pad><pad>',
 '</s>eng_Latn Separation and reintegration continues to make a distinction between the two pools from generation to generation.</s><pad><pad>',
 '</s>eng_Latn Separation and reintegration continues to make a distinction between the two pools with each generation.</s><pad><pad><pad>',
 '</s>eng_Latn Separation and reintegration continues to make a significant difference between the two pools from generation to generation.</s

In [88]:
df[df["task"] == "DM"]

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,A sloping top .,tgt,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",Hallucination,0.6
1,To react too much .,tgt,Please try not to overreact if she drives badl...,To react too much or too intensely .,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.0
2,The process of spoiling ; the state of being s...,tgt,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
3,To arrange in a particular way .,tgt,The way the opposition has framed the argument...,To construct in words so as to establish a con...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",Hallucination,0.6
4,A feeling of concern ; a feeling of anxiety .,tgt,To mix with thy concernments i desist . What i...,That in which one is concerned or interested ;...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",Hallucination,0.6
...,...,...,...,...,...,...,...,...,...
183,Of or pertaining to communism .,tgt,Communistic birds . What is the meaning of com...,Living or having their nests in common .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0
184,The act of treating a disease or condition .,tgt,Lahmann 's unprecedented success proved beyond...,Attempted remediation of a health problem foll...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
185,Able to be collected .,tgt,"He 's owed it to us for six months , but it do...",That is likely to be paid .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Not Hallucinati...",Hallucination,0.8
186,A native or inhabitant of the us state of uson...,tgt,As a usonian of celtic-franco/gallego mexican-...,An inhabitant or citizen of the United States ...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0


In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("ltg/flan-t5-definition-en-base",output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("ltg/flan-t5-definition-en-base")

In [6]:
dm_hal = df[(df["task"] == "DM") & (df["label"] == "Hallucination")]
dm_hal

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,A sloping top .,tgt,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",Hallucination,0.6
2,The process of spoiling ; the state of being s...,tgt,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
3,To arrange in a particular way .,tgt,The way the opposition has framed the argument...,To construct in words so as to establish a con...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",Hallucination,0.6
4,A feeling of concern ; a feeling of anxiety .,tgt,To mix with thy concernments i desist . What i...,That in which one is concerned or interested ;...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",Hallucination,0.6
6,To suffocate .,tgt,Having divested the child he kissed her gently...,To undress .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0
...,...,...,...,...,...,...,...,...,...
183,Of or pertaining to communism .,tgt,Communistic birds . What is the meaning of com...,Living or having their nests in common .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0
184,The act of treating a disease or condition .,tgt,Lahmann 's unprecedented success proved beyond...,Attempted remediation of a health problem foll...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",Hallucination,0.6
185,Able to be collected .,tgt,"He 's owed it to us for six months , but it do...",That is likely to be paid .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Not Hallucinati...",Hallucination,0.8
186,A native or inhabitant of the us state of uson...,tgt,As a usonian of celtic-franco/gallego mexican-...,An inhabitant or citizen of the United States ...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Hallucination, Hallucination, ...",Hallucination,1.0


### Visualizing positive example

In [36]:
dm_hal[dm_hal["p(Hallucination)"] == 1].iloc[0].to_dict()

{'hyp': 'To suffocate .',
 'ref': 'tgt',
 'src': 'Having divested the child he kissed her gently and gave her a little pat to make her stand off . What is the meaning of divest ?',
 'tgt': 'To undress .',
 'model': 'ltg/flan-t5-definition-en-base',
 'task': 'DM',
 'labels': ['Hallucination',
  'Hallucination',
  'Hallucination',
  'Hallucination',
  'Hallucination'],
 'label': 'Hallucination',
 'p(Hallucination)': 1.0}

In [37]:
src = dm_hal[dm_hal["p(Hallucination)"] == 1].iloc[0]["src"]
src

'Having divested the child he kissed her gently and gave her a little pat to make her stand off . What is the meaning of divest ?'

In [38]:
tokens = tokenizer(src,return_tensors="pt")
outputs = model.generate(**tokens,
                         return_dict_in_generate=True,
                         output_scores=True,
                         output_attentions=True,
                         output_hidden_states=True)

/Users/ilyalasy/miniconda3/envs/lm/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [39]:
encoder_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
decoder_tokens = tokenizer.convert_ids_to_tokens(outputs.sequences[0])

result = model(input_ids=tokens["input_ids"],decoder_input_ids=outputs.sequences, use_cache=False,                
        output_attentions=True,        
    )

In [ ]:
head_view(
    encoder_attention=result.encoder_attentions,
    decoder_attention=result.decoder_attentions,
    cross_attention=result.cross_attentions,
    encoder_tokens= encoder_tokens,
    decoder_tokens = decoder_tokens
)

### Visualizing negative example

In [47]:
df[(df["p(Hallucination)"] == 0) & (df["task"] == "DM")].iloc[0].to_dict()

{'hyp': 'To react too much .',
 'ref': 'tgt',
 'src': 'Please try not to overreact if she drives badly when she is first learning . What is the meaning of overreact ?',
 'tgt': 'To react too much or too intensely .',
 'model': 'ltg/flan-t5-definition-en-base',
 'task': 'DM',
 'labels': ['Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination'],
 'label': 'Not Hallucination',
 'p(Hallucination)': 0.0}

In [48]:
src = df[(df["p(Hallucination)"] == 0) & (df["task"] == "DM")].iloc[0]["src"]
src


'Please try not to overreact if she drives badly when she is first learning . What is the meaning of overreact ?'

In [49]:
tokens = tokenizer(src,return_tensors="pt")
outputs = model.generate(**tokens,
                         return_dict_in_generate=True,
                         output_scores=True,
                         output_attentions=True,
                         output_hidden_states=True)

/Users/ilyalasy/miniconda3/envs/lm/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [50]:
encoder_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
decoder_tokens = tokenizer.convert_ids_to_tokens(outputs.sequences[0])

result = model(input_ids=tokens["input_ids"],decoder_input_ids=outputs.sequences, use_cache=False,                
        output_attentions=True,        
    )

In [ ]:
head_view(
    encoder_attention=result.encoder_attentions,
    decoder_attention=result.decoder_attentions,
    cross_attention=result.cross_attentions,
    encoder_tokens= encoder_tokens,
    decoder_tokens = decoder_tokens
)

### Seq-Logprob

In [16]:
src = dm_hal[dm_hal["p(Hallucination)"] == 1].iloc[0]["src"]
src

'Having divested the child he kissed her gently and gave her a little pat to make her stand off . What is the meaning of divest ?'

In [17]:
tokens = tokenizer(src,return_tensors="pt")
outputs = model.generate(**tokens,
                         return_dict_in_generate=True,
                         output_scores=True,
                         output_attentions=True,
                         output_hidden_states=True)

/Users/ilyalasy/miniconda3/envs/lm/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [18]:
print(tokenizer.convert_ids_to_tokens(outputs.sequences[0]))
print(tokenizer.decode(outputs.sequences[0]))

['<pad>', '▁To', '▁', 's', 'uff', 'o', 'cate', '▁', '.', '</s>']
<pad> To suffocate.</s>


In [51]:
def get_conf_score(outputs):
    max_logprobs = [score.max() for score in outputs.scores]
    conf_score = sum(max_logprobs) / len(max_logprobs)
    return conf_score

def generate(source):
    tokens = tokenizer(source,return_tensors="pt")
    outputs = model.generate(**tokens,
                            return_dict_in_generate=True,
                            output_scores=True,)
    return get_conf_score(outputs)

def get_token_lengths(sequences):
    tok_lens = []
    for seq in sequences:
        toks = tokenizer.convert_ids_to_tokens(seq,skip_special_tokens=True)
        tok_lens.append(len(toks))
    return torch.tensor(tok_lens)

In [20]:
srcs = dm_hal[dm_hal["p(Hallucination)"] == 1]["src"].values.tolist()

In [52]:
tokens = tokenizer(srcs,return_tensors="pt",padding=True,truncation=True)
outputs = model.generate(**tokens,
                        return_dict_in_generate=True,
                        output_scores=True,)

/Users/ilyalasy/miniconda3/envs/lm/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [53]:
tok_lens = get_token_lengths(outputs.sequences)
#TODO: don't include conf scores from pad tokens
conf_scores = torch.stack([torch.max(score,dim=1).values for score in outputs.scores]).sum(0)  / tok_lens
conf_scores

tensor([ 9.4071,  5.3401, -0.2419,  0.8012,  5.9969,  4.8076,  4.0492,  3.5485,
         6.0472,  4.6019,  6.1943,  4.1186,  7.5194,  5.5130,  3.0846,  4.9129,
         4.3745,  4.8689,  3.2190,  7.8033, 10.2741,  7.3097,  4.6601,  7.4027,
         6.3553,  3.5634,  5.4295, 10.4581,  5.3850,  8.9073,  6.4095,  4.3153,
         6.9461,  5.1326,  4.2383])

In [54]:
print(conf_scores.min())
print(conf_scores.max())

tensor(-0.2419)
tensor(10.4581)


In [55]:
srcs = df[(df["p(Hallucination)"] == 0) & (df["task"] == "DM")]["src"].values.tolist()

In [56]:
tokens = tokenizer(srcs,return_tensors="pt",padding=True,truncation=True)
outputs = model.generate(**tokens,
                        return_dict_in_generate=True,
                        output_scores=True,)

In [59]:
tok_lens = get_token_lengths(outputs.sequences)
conf_scores = torch.stack([torch.max(score,dim=1).values for score in outputs.scores]).sum(0) / tok_lens
conf_scores

tensor([ 9.4396,  3.7341,  5.8660,  4.8929,  2.1771, 13.0546,  8.6266,  4.7140,
         3.3716,  5.6011, 17.0202,  5.0676,  4.2212, 11.1073,  4.7932,  6.5653,
        24.7344,  5.5094,  7.6758,  6.1724,  3.6417,  5.7009,  8.1423, 13.5789,
         6.1132,  7.8263, 32.1291,  5.4459,  4.7539,  4.6217,  6.5457])

In [60]:
print(conf_scores.min())
print(conf_scores.max())


tensor(2.1771)
tensor(32.1291)


### Hidden States Classifier

In [30]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
path_dev = "../data/labeled-train.model-aware.v2.json"
df = pd.read_json(path_dev)

In [4]:
TASK = "DM"
FILTER = False

task_df = df[(df["task"] == TASK)] 
model_name = task_df["model"].unique().item()
if FILTER:
    task_df = task_df[(task_df["p(Hallucination)"] == 0) | (task_df["p(Hallucination)"]==1)]
task_df = task_df[["src","label"]]
task_df["label"] = task_df["label"].astype('category')

In [5]:
# Check that MPS is available
if torch.cuda.is_available():
    device = "cuda"
elif not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    device = "cpu"
else:
    device = "mps"
print(device)

cuda


In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
batch_size = 32 # turned out to be the best size, larger ones result in worse detections
batched_df = [task_df[i:i+batch_size] for i in range(0,len(task_df),batch_size)]

In [19]:
def get_hidden_states(outputs):
    eos_indices = []
    for seq in outputs.sequences:
        nonzero = (seq == tokenizer.eos_token_id).nonzero()
        if nonzero.nelement() != 0:
            idx = nonzero.item()
        else:
            idx = seq.size(0)-1 #last token
        eos_indices.append(idx-1) #skip first pad token    
    eos_indices = torch.tensor(eos_indices,device=device)

    decoder_hidden_states = []
    for t in outputs.decoder_hidden_states:
        decoder_hidden_states.append(torch.stack(t))
    decoder_hidden_states = torch.stack(decoder_hidden_states)
    seq_len,layers,batch, _, hidden_dim = decoder_hidden_states.size()
    decoder_hidden_states = decoder_hidden_states.reshape((layers,batch,seq_len,-1))

    eos_indices = eos_indices.reshape((1,-1,1,1)).repeat(layers,1,1,hidden_dim)
    hiddens = decoder_hidden_states.gather(dim=2,index=eos_indices).squeeze(2)
    return hiddens


In [6]:
with open("../data/DM-train.pkl", "rb") as f:
    hiddens,labels = pickle.load(f)

# start_batch = 0
# dataset = []
# for batch in tqdm.tqdm(batched_df[start_batch:],total=len(batched_df[start_batch:])):
#     tokens = tokenizer(batch["src"].to_list(),
#                     return_tensors="pt",
#                     padding=True,
#                     truncation=True).to(device)
#     with torch.no_grad():
#         outputs = model.generate(**tokens,
#                                 max_new_tokens=25,
#                                 return_dict_in_generate=True,
#                                 output_hidden_states=True)
#     hiddens = get_hidden_states(outputs)
#     dataset.append((hiddens.to("cpu"),batch["label"].cat.codes.values))
# hiddens,labels = list(zip(*dataset))
# hiddens = torch.cat(hiddens,dim=1)
# labels = np.concatenate(labels)
# with open("hidden.pkl", "wb") as f:
#     pickle.dump((hiddens,labels),f)

In [13]:
# hiddens[layer_num].size()
labels = np.concatenate(labels)

In [17]:
hidden_dim = hiddens.size(-1)
class StatesClassifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.ln1 = nn.Linear(hidden_dim,256)
        self.ln2 = nn.Linear(256,128)
        self.ln3 = nn.Linear(128,64)
        self.head = nn.Linear(64,1)

    def forward(self,x):
        x = self.ln1(x).relu()
        x = self.ln2(x).relu()
        x = self.ln3(x).relu()
        return self.head(x).sigmoid()

In [18]:
classifier = StatesClassifier().to(device)

In [19]:
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.0001)

In [28]:
def train_one_epoch(epoch_index,loader):
    running_loss = 0.
    
    for i, data in enumerate(loader):        
        inputs, labels = data
        labels = labels.unsqueeze(1).float()
        optimizer.zero_grad()

        outputs = classifier(inputs.to(device))

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels.to(device))
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()        

    return running_loss / len(loader)

In [33]:
EPOCHS = 20

best_vloss = 1_000_000.

model_path = None

total_layers = hiddens.size(0)
results = []
for layer_num in range(total_layers):
    writer = SummaryWriter(f'runs/hiddens_layer_{layer_num}')
    train_x, val_x, train_y,val_y = train_test_split(hiddens[layer_num],labels,test_size=0.1,random_state=42,stratify=labels)
    training_loader = torch.utils.data.DataLoader(list(zip(train_x,train_y)), batch_size=32, shuffle=True)
    validation_loader = torch.utils.data.DataLoader(list(zip(val_x,val_y)), batch_size=32, shuffle=False)

    for epoch_number in range(EPOCHS):
        print(f'EPOCH {epoch_number+1}:')

        # Make sure gradient tracking is on, and do a pass over the data
        classifier.train(True)
        avg_loss = train_one_epoch(epoch_number,training_loader)

        running_vloss = 0.0
        running_vacc = 0.0

        classifier.eval()
        with torch.no_grad():
            for i, vdata in enumerate(validation_loader):
                vinputs, vlabels = vdata
                vinputs= vinputs.to(device)
                vlabels = vlabels.unsqueeze(1).float().to(device)

                voutputs = classifier(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                running_vloss += vloss
                running_vacc += (voutputs.round() == vlabels).float().mean()

        avg_vloss = running_vloss / (i + 1)
        avg_acc = running_vacc / (i+1)
        print(f'LOSS train {avg_loss} valid {avg_vloss}')
        print(f'Valid ACC {avg_acc}')

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                        { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch_number + 1)
        writer.add_scalars('Validation accuracy',
                        { 'ACC' : avg_acc},
                        epoch_number + 1)
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss    
            model_path = f'model_{epoch_number}_{best_vloss}'
            # torch.save(model.state_dict(), model_path)
            results.append((best_vloss,avg_acc,epoch_number,layer_num))


        epoch_number += 1
print(list(sorted(results,key=lambda x:x[0]))[:10])

EPOCH 1:
LOSS train 1.809154839486095 valid 1.3699495792388916
Valid ACC 0.6865234375
EPOCH 2:
LOSS train 1.6973109913210498 valid 1.3730589151382446
Valid ACC 0.6884765625
EPOCH 3:
LOSS train 1.6937941563256242 valid 1.2456235885620117
Valid ACC 0.6904296875
EPOCH 4:
LOSS train 1.6844726184792553 valid 1.2128487825393677
Valid ACC 0.6875
EPOCH 5:
LOSS train 1.6228884999633681 valid 1.2178575992584229
Valid ACC 0.685546875
EPOCH 6:
LOSS train 1.6259956847057275 valid 1.221069097518921
Valid ACC 0.6884765625
EPOCH 7:
LOSS train 1.6320871089367157 valid 1.2257506847381592
Valid ACC 0.6796875
EPOCH 8:
LOSS train 1.665392702669962 valid 1.2318390607833862
Valid ACC 0.68359375
EPOCH 9:
LOSS train 1.4847571821288859 valid 0.8308987617492676
Valid ACC 0.69140625
EPOCH 10:
LOSS train 0.6512302348377011 valid 0.6673679947853088
Valid ACC 0.6787109375
EPOCH 11:
LOSS train 0.6321208630259155 valid 0.684244692325592
Valid ACC 0.6728515625
EPOCH 12:
LOSS train 0.6296492290835008 valid 0.68606120347

In [34]:
list(sorted(results,key=lambda x:x[0]))[:10]

[(tensor(0.6293, device='cuda:0'), tensor(0.6895, device='cuda:0'), 0, 5),
 (tensor(0.6405, device='cuda:0'), tensor(0.6816, device='cuda:0'), 0, 2),
 (tensor(0.6618, device='cuda:0'), tensor(0.6836, device='cuda:0'), 4, 1),
 (tensor(0.6674, device='cuda:0'), tensor(0.6787, device='cuda:0'), 9, 0),
 (tensor(0.8309, device='cuda:0'), tensor(0.6914, device='cuda:0'), 8, 0),
 (tensor(1.2128, device='cuda:0'), tensor(0.6875, device='cuda:0'), 3, 0),
 (tensor(1.2456, device='cuda:0'), tensor(0.6904, device='cuda:0'), 2, 0),
 (tensor(1.3699, device='cuda:0'), tensor(0.6865, device='cuda:0'), 0, 0)]